In [1]:
import pandas as pd
import numpy  as np
import plotly 
import plotly.io as pio
import re

import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input,  State
from dash.dependencies import Input, Output

from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
from zipfile import ZipFile

import yfinance as yf
from yahoo_fin import stock_info as si

import os

# Self created packages

# from database import Database

**End State**
1. Dashboard that pulls live from a website once a day using Amazon Web Servies
2. S3 database, Jobs that call once a day, Dashboard full workflow

**Load Data**

In [2]:
def load_data():
	'''
	'''
	with ZipFile('stock-time-series-20050101-to-20171231.zip') as zp:

		# add logic to pull stock name into data key
		data = {}

		for files in zp.namelist():
			name  = files.replace('.csv', '')

			df    = pd.read_csv(zp.open(files))
			data[name] = df

	return data

# data        = load_data()
# default_col = 'AAPL_2006-01-01_to_2018-01-01'

In [3]:
# Scrape the list of S&P 500 companies from Wikipedia
url         = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url, header=0)[0]

# Extract the ticker symbols
tickers     = sp500_table['Symbol'].tolist()

# Function to get stock data for multiple companies
def get_multiple_stock_data(tickers, period='1y', interval='1d'):
    data       = {}
    news       = {}
    earnings   = {}
    financials = {}
    for ticker in tickers:
        stock        = yf.Ticker(ticker)
        stock_news   = stock.news
        
        data[ticker] = stock.history(period=period, interval=interval)
        earning      = stock.balance_sheet
        financial    = stock.financials
        data[ticker] = data[ticker].reset_index()
        data[ticker].columns = [col if col != 'Datetime' else 'Date' for col in data[ticker].columns]

        news[ticker]       = stock_news
        earnings[ticker]   = earning
        financials[ticker] = financial
        
    return data, news, earnings, financials


period_options   = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']

interval_options = ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo']

period_intervals = {'Day': ('1d', '15m'), 'Month':('1mo', '1d'), 'Year':('1y', '1d'),
                    '5 Years':('5y', '1wk'), 'Max':('max', '1mo')}

data_dictionary  = {}
for key, pair_option in period_intervals.items():
    period_option                    = pair_option[0]
    interval_option                  = pair_option[1]
    data, news, earnings, financials = get_multiple_stock_data(tickers[:20], period = period_option, interval = interval_option)
    data_dictionary[key]    = data


In [4]:
news['MMM']

[{'uuid': 'ebf0348c-6a56-36cb-b403-1df489e74ec4',
  'title': 'Post-it Maker 3M Raises Annual Outlook After Q2 Beat, Stock Soars',
  'publisher': 'Benzinga',
  'link': 'https://finance.yahoo.com/news/post-maker-3m-raises-annual-140021305.html',
  'providerPublishTime': 1722175221,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/BrJck.9IaYIVKYX9VAGgKw--~B/aD00ODA7dz03MjA7YXBwaWQ9eXRhY2h5b24-/https://media.zenfs.com/en/benzinga_79/f60b88f650086c2c3ae6ead0b73cbd5c',
     'width': 720,
     'height': 480,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/a0lg4QclMm2X8SmPW52p7A--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/benzinga_79/f60b88f650086c2c3ae6ead0b73cbd5c',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['MMM']},
 {'uuid': 'e7916db6-4728-3460-ac24-c41be70a8c1f',
  'title': 'Q2 2024 3M Co Earnings Call',
  'publisher': 'Thomson Reuters

In [28]:
data_dictionary['Day']['MMM']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-07-19 09:30:00-04:00,104.750000,104.750000,103.489998,103.535004,286460,0.0,0.0
1,2024-07-19 09:45:00-04:00,103.489998,103.869499,103.129997,103.760002,64481,0.0,0.0
2,2024-07-19 10:00:00-04:00,103.760002,104.440002,103.720001,104.014999,92685,0.0,0.0
3,2024-07-19 10:15:00-04:00,103.980003,104.084999,103.470100,103.510002,57169,0.0,0.0
4,2024-07-19 10:30:00-04:00,103.500000,104.379997,103.459999,104.330002,103153,0.0,0.0
5,2024-07-19 10:45:00-04:00,104.320000,104.467499,103.695000,103.930000,76515,0.0,0.0
6,2024-07-19 11:00:00-04:00,103.930000,104.105003,103.709999,103.714996,91709,0.0,0.0
7,2024-07-19 11:15:00-04:00,103.750000,104.214996,103.730003,103.754997,68856,0.0,0.0
8,2024-07-19 11:30:00-04:00,103.760002,103.980003,103.676903,103.800003,61019,0.0,0.0
9,2024-07-19 11:45:00-04:00,103.800003,104.250000,103.760002,104.230003,59713,0.0,0.0


In [29]:
counter = 0 
for key in news.keys():
	news[key] = pd.DataFrame(news[key])
	news[key] = news[key].drop(columns = ['thumbnail', 'uuid'])
	if counter > 2:
		break
	
	counter += 1

In [41]:
news['MMM']

,title,publisher,link,providerPublishTime,type,relatedTickers
0,Is 3M a Millionaire Maker?,Motley Fool,https://finance.yahoo.com/m/dd617720-d961-3c66...,1721462580,STORY,[MMM]
1,3M (MMM) Expected to Beat Earnings Estimates: ...,Zacks,https://finance.yahoo.com/news/3m-mmm-expected...,1721397619,STORY,"[HON, MMM]"
2,GE Aerospace (GE) to Report Q2 Earnings: What ...,Zacks,https://finance.yahoo.com/news/ge-aerospace-ge...,1721396940,STORY,[MMM]
3,3M (MMM) Rises 13.4% in 3 Months: How Should Y...,Zacks,https://finance.yahoo.com/news/3m-mmm-rises-13...,1721312700,STORY,[MMM]
4,3M (MMM) Ascends While Market Falls: Some Fact...,Zacks,https://finance.yahoo.com/news/3m-mmm-ascends-...,1721252720,STORY,[MMM]
5,3M invests in green hydrogen leader Ohmium in ...,PR Newswire,https://finance.yahoo.com/news/3m-invests-gree...,1721142000,STORY,[MMM]
6,Three US Stocks Estimated To Be Trading Below ...,Simply Wall St.,https://finance.yahoo.com/news/three-us-stocks...,1720782216,STORY,"[MMM, LAAOF, LI, BMRN]"
7,The 3 Most Undervalued Dow Stocks to Buy in Ju...,InvestorPlace,https://finance.yahoo.com/news/3-most-underval...,1720717200,STORY,"[^DJI, MMM]"


In [43]:
financials['MMM'].transpose().reset_index()

,index,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBITDA,...,Operating Expense,Research And Development,Selling General And Administration,General And Administrative Expense,Other Gand A,Salaries And Wages,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
0,2023-12-31,10008000.0,0.278,-6795000000.0,36000000.0,36000000.0,-6995000000.0,1987000000.0,18477000000.0,-6759000000.0,...,23239000000.0,1842000000.0,21397000000.0,21397000000.0,21526000000.0,-129000000.0,14204000000.0,18477000000.0,32681000000.0,32681000000.0
1,2022-12-31,235488000.0,0.096,6232000000.0,2453000000.0,2453000000.0,5777000000.0,1831000000.0,19232000000.0,8685000000.0,...,10663000000.0,1862000000.0,8801000000.0,8801000000.0,9049000000.0,-248000000.0,14997000000.0,19232000000.0,34229000000.0,34229000000.0
2,2021-12-31,0.0,0.178,9607000000.0,0.0,0.0,5921000000.0,1915000000.0,18795000000.0,9607000000.0,...,8894000000.0,1994000000.0,6900000000.0,6900000000.0,7197000000.0,-297000000.0,16560000000.0,18795000000.0,35355000000.0,35355000000.0
3,2020-12-31,76633000.0,0.197,8846000000.0,389000000.0,389000000.0,5449000000.0,1911000000.0,16605000000.0,9235000000.0,...,8673000000.0,1878000000.0,6795000000.0,6795000000.0,6929000000.0,-134000000.0,15579000000.0,16605000000.0,32184000000.0,32184000000.0
4,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
earnings['MMM'].transpose().reset_index().columns

Index(['index', 'Treasury Shares Number', 'Ordinary Shares Number',
       'Share Issued', 'Net Debt', 'Total Debt', 'Tangible Book Value',
       'Invested Capital', 'Working Capital', 'Net Tangible Assets',
       'Capital Lease Obligations', 'Common Stock Equity',
       'Total Capitalization', 'Total Equity Gross Minority Interest',
       'Minority Interest', 'Stockholders Equity',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Treasury Stock', 'Retained Earnings',
       'Additional Paid In Capital', 'Capital Stock', 'Common Stock',
       'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Employee Benefits',
       'Non Current Pension And Other Postretirement Benefit Plans',
       'Tradeand Other Payables Non Current',
       'Non Current Deferred Liabilities',
       'Non Current Deferred Taxes Liabilities',
       'Long Term Debt And Capi

In [60]:
type(earnings['MMM'].transpose()[col].iloc[0])

float

In [61]:
for col in earnings['MMM'].transpose().columns:
	print(type(earnings['MMM'].transpose()[col].iloc[0]))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class '

In [45]:
dummy = earnings['MMM'].transpose().reset_index()
dummy.columns = [col if col!= 'index' else 'Date' for col in dummy.columns ]
dummy

,Date,Treasury Shares Number,Ordinary Shares Number,Share Issued,Net Debt,Total Debt,Tangible Book Value,Invested Capital,Working Capital,Net Tangible Assets,...,Work In Process,Raw Materials,Receivables,Other Receivables,Accounts Receivable,Allowance For Doubtful Accounts Receivable,Gross Accounts Receivable,Cash Cash Equivalents And Short Term Investments,Other Short Term Investments,Cash And Cash Equivalents
0,2023-12-31,391451920.0,552581136.0,944033056.0,10102000000.0,16854000000.0,-12346000000.0,20842000000.0,1082000000.0,-12346000000.0,...,1424000000.0,1105000000.0,4864000000.0,114000000.0,4750000000.0,-141000000.0,4891000000.0,5986000000.0,53000000.0,5933000000.0
1,2022-12-31,394787951.0,549245105.0,944033056.0,12284000000.0,16855000000.0,-2767000000.0,30661000000.0,5165000000.0,-2767000000.0,...,1606000000.0,1269000000.0,4635000000.0,103000000.0,4532000000.0,-174000000.0,4706000000.0,3893000000.0,238000000.0,3655000000.0
2,2021-12-31,372187578.0,571845478.0,944033056.0,12799000000.0,18310000000.0,-3728000000.0,32409000000.0,6368000000.0,-3728000000.0,...,1577000000.0,1212000000.0,4770000000.0,110000000.0,4660000000.0,-189000000.0,4849000000.0,4765000000.0,201000000.0,4564000000.0
3,2020-12-31,366283418.0,577749638.0,944033056.0,14161000000.0,19753000000.0,-6770000000.0,31662000000.0,7034000000.0,-6770000000.0,...,1226000000.0,932000000.0,4830000000.0,125000000.0,4705000000.0,-233000000.0,4938000000.0,5038000000.0,404000000.0,4634000000.0
4,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
news[key]

,title,publisher,link,providerPublishTime,type,relatedTickers
0,What To Expect in the Markets This Week,Investopedia,https://finance.yahoo.com/m/86aee722-448d-36b8...,1721559600,STORY,"[TSLA, GOOG, ABBV]"
1,AbbVie (ABBV) Rises As Market Takes a Dip: Key...,Zacks,https://finance.yahoo.com/news/abbvie-abbv-ris...,1721425519,STORY,[ABBV]
2,"Tesla, Alphabet, Visa earnings, June PCE data:...",Yahoo Finance Video,https://finance.yahoo.com/video/tesla-alphabet...,1721424828,VIDEO,"[KO, VZ, STLA, ABBV, V, CMCSA, GOOG, TSLA, IBM..."
3,Boehringer Ingelheim offers Humira biosimilar ...,Pharmaceutical Technology,https://finance.yahoo.com/m/d149db51-5b24-3754...,1721402019,STORY,"[ABBV, GDRX, CVS]"
4,"Pharma Stock Roundup: JNJ, NVS Q2 Earnings, RH...",Zacks,https://finance.yahoo.com/news/pharma-stock-ro...,1721395560,STORY,"[NVS, ABBV, ROG.SW]"
5,Investors in AbbVie (NYSE:ABBV) have seen stro...,Simply Wall St.,https://finance.yahoo.com/news/investors-abbvi...,1721394026,STORY,[ABBV]
6,Earnings Preview: AbbVie (ABBV) Q2 Earnings Ex...,Zacks,https://finance.yahoo.com/news/earnings-previe...,1721311283,STORY,[ABBV]
7,Boehringer cuts price of Humira biosimilar in ...,BioPharma Dive,https://finance.yahoo.com/m/f76fdc97-ca9e-398a...,1721302980,STORY,[ABBV]


In [9]:
news['MMM']

[{'uuid': 'dd617720-d961-3c66-b0be-9988d65f6ca4',
  'title': 'Is 3M a Millionaire Maker?',
  'publisher': 'Motley Fool',
  'link': 'https://finance.yahoo.com/m/dd617720-d961-3c66-b0be-9988d65f6ca4/is-3m-a-millionaire-maker%3F.html',
  'providerPublishTime': 1721462580,
  'type': 'STORY',
  'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/sXf3kLS2RkF5Pm9JPCCrgA--~B/aD05MzM7dz0xNDAwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/dae4dd40c7c52f64e3cb535da15506e9',
     'width': 1400,
     'height': 933,
     'tag': 'original'},
    {'url': 'https://s.yimg.com/uu/api/res/1.2/RNC_yUsR.ehogid43Lsp.Q--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/motleyfool.com/dae4dd40c7c52f64e3cb535da15506e9',
     'width': 140,
     'height': 140,
     'tag': '140x140'}]},
  'relatedTickers': ['MMM']},
 {'uuid': 'fedaf7f3-17c4-3a59-90a3-c6715b714af2',
  'title': '3M (MMM) Expected to Beat Earnings Estimates: What to Know Ahead 

In [4]:
data_dictionary['Day']['MMM']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-07-19 09:30:00-04:00,104.750000,104.750000,103.489998,103.535004,286460,0.0,0.0
1,2024-07-19 09:45:00-04:00,103.489998,103.869499,103.129997,103.760002,64481,0.0,0.0
2,2024-07-19 10:00:00-04:00,103.760002,104.440002,103.720001,104.014999,92685,0.0,0.0
3,2024-07-19 10:15:00-04:00,103.980003,104.084999,103.470100,103.510002,57169,0.0,0.0
4,2024-07-19 10:30:00-04:00,103.500000,104.379997,103.459999,104.330002,103153,0.0,0.0
5,2024-07-19 10:45:00-04:00,104.320000,104.467499,103.695000,103.930000,76515,0.0,0.0
6,2024-07-19 11:00:00-04:00,103.930000,104.105003,103.709999,103.714996,91709,0.0,0.0
7,2024-07-19 11:15:00-04:00,103.750000,104.214996,103.730003,103.754997,68856,0.0,0.0
8,2024-07-19 11:30:00-04:00,103.760002,103.980003,103.676903,103.800003,61019,0.0,0.0
9,2024-07-19 11:45:00-04:00,103.800003,104.250000,103.760002,104.230003,59713,0.0,0.0


In [4]:
# db = Database()
# db.insert_stock_prices(data_dictionary['Max'])

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'Date' in 'field list'
[SQL: INSERT INTO lifetime_stocks (`Date`, `Open`, `High`, `Low`, `Close`, `Volume`, `Dividends`, `Stock Splits`) VALUES (%(Date)s, %(Open)s, %(High)s, %(Low)s, %(Close)s, %(Volume)s, %(Dividends)s, %(Stock_Splits)s)]
[parameters: [{'Date': datetime.datetime(1962, 2, 1, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>), 'Open': 0.0, 'High': 0.42587024857359296, 'Low': 0.3948013868809007, 'Close': 0.40465253591537476, 'Volume': 3408123, 'Dividends': 0.0125, 'Stock_Splits': 0.0}, {'Date': datetime.datetime(1962, 3, 1, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>), 'Open': 0.0, 'High': 0.4266525172912799, 'Low': 0.3954711309074974, 'Close': 0.42513152956962585, 'Volume': 4081711, 'Dividends': 0.0, 'Stock_Splits': 0.0}, {'Date': datetime.datetime(1962, 4, 1, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>), 'Open': 0.0

In [10]:
# db.show_tables(show_rows = True)

(1, 'MMM')
(2, 'AOS')
(3, 'ABT')
(4, 'ABBV')
(5, 'ACN')
(6, 'ADBE')
(7, 'AMD')
(8, 'AES')
(9, 'AFL')
(10, 'A')
(11, 'APD')
(12, 'ABNB')
(13, 'AKAM')
(14, 'ALB')
(15, 'ARE')
(16, 'ALGN')
(17, 'ALLE')
(18, 'LNT')
(19, 'ALL')
(20, 'GOOGL')


In [8]:
def convert_timestamps(df):
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):

            df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M')  # Converting to string format

    return df

# for day in data_dictionary:
#     for company in data_dictionary[day]:
#         data_dictionary[day][company] = convert_timestamps(data_dictionary[day][company])

In [6]:
# App(data_dictionary, news, earnings, financials)

In [30]:
pd.DataFrame(news['MMM'])

,uuid,title,publisher,link,providerPublishTime,type,thumbnail,relatedTickers
0,d8da651d-d809-3070-a9df-30a0f2545f20,1 Wall Street Analyst Thinks 3M Stock Is Going...,Motley Fool,https://finance.yahoo.com/m/d8da651d-d809-3070...,1718124054,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[MMM]
1,75062ef6-14ef-3714-917c-92e988925622,How 3M Is Innovating To Address Climate Challe...,ACCESSWIRE,https://finance.yahoo.com/news/3m-innovating-a...,1718108100,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[MMM]
2,94ee34d5-59fb-3d4b-a6bc-fef547c373db,3M (MMM) Stock Slides as Market Rises: Facts t...,Zacks,https://finance.yahoo.com/news/3m-mmm-stock-sl...,1718055917,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[MMM]
3,ef3765c5-c8e5-31f9-a9d9-3aa3a35458af,3M Company (NYSE:MMM) Upgraded by Bank of America,Insider Monkey,https://finance.yahoo.com/news/3m-company-nyse...,1718054453,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[MMM]
4,f6372aeb-89d8-3ee2-b416-d7b30d5090ef,3M Named Official Respiratory Protection Spons...,ACCESSWIRE,https://finance.yahoo.com/news/3m-named-offici...,1718021700,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,[MMM]
5,f74760d5-cb90-466e-bb12-b990b908cbd3,Most CEOs are defeating attempts to vote down ...,Yahoo Finance,https://finance.yahoo.com/news/most-ceos-are-d...,1718010250,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,"[^IXIC, ^GSPC, AZN, AZNCF, BLK, ^DJI, TSLA, MMM]"
6,dd3eb0ff-f3a5-3d1c-aed7-d4e7de500c7b,Industrial moves for the AI boom: Top picks,Yahoo Finance Video,https://finance.yahoo.com/video/industrial-mov...,1717792419,VIDEO,{'resolutions': [{'url': 'https://s.yimg.com/u...,"[NVT, XYL, HTHIF, HTHIY, ETN, MMM]"
7,b5e8e9c0-0f51-3bdd-9a50-256e2b7ff741,Blue-Chip Steals: 3 Top-Tier Stocks Trading at...,InvestorPlace,https://finance.yahoo.com/news/blue-chip-steal...,1717606800,STORY,{'resolutions': [{'url': 'https://s.yimg.com/u...,"[ADM, MMM, T]"


In [96]:
class App:
    def __init__(self, data_dict, news, earnings, financials):
        self.app = dash.Dash(__name__)
        self.data_dict = data_dict
        self.news_dict = news
        self.earnings_dict  = earnings
        self.financial_dict = financials
        self.news_cols      = ['title', 'publisher']
        
        self.news     = pd.DataFrame(news[list(earnings.keys())[0]])
        self.data     = data_dict[list(data_dict.keys())[0]]
        self.earnings = earnings[list(earnings.keys())[0]]
        self.financials = financials[list(financials.keys())[0]]
        self.default_col = list(self.data.keys())[0]
        self.periods = list(self.data_dict.keys())

        self.stock_columns = self.data[self.default_col].columns
        self.display_data  = self.data[self.default_col]
        self.companies     = self.get_companies(self.data)
        
        self.company   = self.companies[0]
        self.news = self.convert_type(self.news, ['thumbnail', 'link', 'relatedTickers'])[self.news_cols]
        self.app_layout()
        self.run_server()

    def update_periods(self, data_dict):
        ''' Update the default stock data periods '''
        time_periods = list(self.data_dict.keys())
        if time_periods != self.periods:
            periods = time_periods
        return periods

    def update_period(self, period):
        ''' Update the data to have a different time period '''
        return self.data_dict[period]

    def update_company(self, company):
        self.company = company
        return self.data[company]

    def get_companies(self, data):
        return list(data.keys())
    
    def run_server(self):
        self.app.run_server(debug=True)  

    def convert_type(self, x, col):
        if isinstance(x, list):
            for c in col:
                x[c] = x[c].astype(str)
        else:
            x[col] = x[col].astype(str)
        return x

    def update_news_data(self, company):
        
        self.news  = pd.DataFrame(self.news_dict[company])
        self.news  = self.convert_type(self.news, ['thumbnail', 'link', 'relatedTickers'])[self.news_cols]
        self.news  = self.news.to_dict('records')

        return self.news

    def app_layout(self):
        self.app.layout = html.Div([
            html.Div([
                html.H1('Stock Monitoring Dashboard')
            ], style={'textAlign': 'center'}),

            html.Div([
                html.Div([
                    dcc.Tabs(
                        id='tabs',
                        value='page-1',
                        vertical=True,
                        children=[
                            dcc.Tab(label='Stock History', value='page-1', children = self.render_page_1()),
                            dcc.Tab(label='Model Forecasts', value='page-2', children = self.render_page_2()),
                            dcc.Tab(label='Relevant News', value='page-3', children = self.render_page_3())
                        ],
                        style={
                            'height': '100vh',
                            'borderRight': '1px solid #d6d6d6',
                            'padding': '10px'
                        }
                    )
                ], style={'width': '20%', 'display': 'inline-block', 'verticalAlign': 'top'}),

                html.Div(id='page-content', style={'width': '75%', 'display': 'inline-block', 'padding': '10px'})
            ])
        ])

        # Initialize the layout with the default tab's content
        self.app.layout.children[-1].children[-1].children = self.render_page_1()

        @self.app.callback(Output('page-content', 'children'),
                           [Input('tabs', 'value')])
        
        # Insert some code to make sure page 1 doesn't return to original state
        # Make sure all states stay in the dashboard and transfer between pages. 
        def render_content(tab):
            if tab == 'page-1':
                return self.render_page_1()
            elif tab == 'page-2':
                return self.render_page_2()
            elif tab == 'page-3':
                return self.render_page_3()
            
        # Callback to update the content of the news table on page 3
        @self.app.callback(
            Output('news-table', 'data'),
            [Input('company-dropdown', 'value')],
            [State('tabs', 'value')]  # Include the state of the tabs to check if page 3 is active
        )

        def update_news_table(selected_company, current_tab):
            if current_tab != 'page-3' or not hasattr(self, 'news_initialized'):
                raise PreventUpdate  # Prevents the callback from executing

            return self.update_news_data(selected_company)

        @self.app.callback(
            # Output('news-table', 'data'), 
            Output('interactive-graph', 'figure'),
            [Input('company-dropdown', 'value'),
             Input('data-dropdown', 'value'),
             Input('period', 'value')]
        )
        def update_graph(selected_company, selected_data, period):
            self.data = self.update_period(period)
            display_data = self.update_company(selected_company)
            self.company
            fig = px.line(display_data, x='Date', y=selected_data, title=f'{selected_company} - {selected_data}', template='plotly_dark')
            return fig

    def render_page_1(self):
        return html.Div([
            html.Label('Select Company'),
            dcc.Dropdown(
                id='company-dropdown',
                options=[{'label': company, 'value': company} for company in self.companies],
                value=self.company
            ),
            html.Br(),
            html.Label('Stock Period'),
            dcc.Dropdown(
                id='period',
                options=[{'label': period, 'value': period} for period in self.periods],
                value=self.periods[0]
            ),
            html.Br(),
            html.Label('Select Data'),
            dcc.Dropdown(
                id='data-dropdown',
                options=[
                    {'label': category, 'value': category} for category in self.stock_columns[1:5]
                ],
                value='Close'
            ),
            dcc.Graph(id='interactive-graph')
        ])

    def render_page_2(self):
        return html.Div([
            html.H3('Earnings'),
            dash_table.DataTable(
                id='earnings-table',
                columns=[{'name': i, 'id': i} for i in self.earnings.columns],
                data=self.earnings.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left'},
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                },
                style_as_list_view=True,
                style_data={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'lineHeight': '15px'
                }
            )
        ])

    def render_page_3(self):

        if not hasattr(self, 'news_initialized'):
            # Initialize or load data for news table
            self.news = self.update_news_data(self.company)
            self.news_initialized = True

        return html.Div([
            html.H3('News'),
            dash_table.DataTable(
                id='news-table',
                columns=[{'name': i, 'id': i} for i in self.news.columns],
                data=self.news.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_cell={'textAlign': 'left'},
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold'
                },
                style_as_list_view=True,
                style_data={
                    'whiteSpace': 'normal',
                    'height': 'auto',
                    'lineHeight': '15px'
                }
            )
        ])

In [97]:
if __name__ == '__main__':
    App(data_dictionary, news, earnings, financials)

    

AttributeError: 'list' object has no attribute 'columns'